## In this part of our study, we are exploring different biomass composition in PlantCoreMetabolism model. By referring to the compositions described in [Yuan et al.,2016](https://doi.org/10.3389/fpls.2016.00537).

##Applying the Biomass composition from 3 different models by -[ Poolman et al.,2009](https://pmc.ncbi.nlm.nih.gov/articles/PMC2773075/); [de Oliveira Dal'Molin et al., 2010](https://doi.org/10.1104/pp.109.148817); and [Arnold & Nikoloski.,2014](https://doi.org/10.1104/pp.114.235358).

#Install cobrapy

In [ ]:
%pip install cobra --quiet

#Import functions required to read sbml and perform pFBA

In [ ]:
from cobra.io import read_sbml_model
from cobra.flux_analysis import pfba


#Import the models from github  ---- Instead of loading from drive  
##created three new models by applying

*   biomass composition from AraCore model by Arnold & Nikoloski, 2014. saved it as "Core_AraCore" in sbml format
*   biomass composition from AraCore model by de Oliveira Dal'Molin et al., 2010. saved it as "Core_AraGEM" in sbml format
*   biomass composition from AraCore model by  Poolman et al.,2009. saved it as "Core_AraMeta" in sbml format

    

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Function to make C3 model
## diel_biomass redefined within the function by replacing Phloem_output with  AraCore_Biomass

In [ ]:
########################################################
#This function was used to set up a C3 leaf diel model #
########################################################
def setupC3DielModel(core_model,transferMets="",starch_sucrose_ratio=None):
    '''
    This function can be used to generate a fully constrained diel C3 leaf model
    from a core model.
    Inputs: 1) a cobra model 2) a list of metabolites allowed to accumulate 3)
    starch to sucrose accumulation rate ratio
    Outputs: a fully constrained diel C3 leaf model
    '''
    from cobra.core import Metabolite, Reaction
    import re

    #create two copies of model elements for day and night
    tempCompDict = dict()
    for comp in core_model.compartments:
        tempCompDict[comp+"1"] = core_model.compartments[comp]+" day"
        tempCompDict[comp+"2"] = core_model.compartments[comp]+" night"

    cobra_model2 = core_model.copy()
    for met in cobra_model2.metabolites:
        met.id = met.id+"1"
        met.name = met.name+" Day"
        met.compartment = met.compartment+"1"
    for rxn in cobra_model2.reactions:
        rxn.id = rxn.id+"1"
        rxn.name = rxn.name+" Day"

    cobra_model3 = core_model.copy()
    for met in cobra_model3.metabolites:
        met.id = met.id+"2"
        met.name = met.name+" Night"
        met.compartment = met.compartment+"2"
    for rxn in cobra_model3.reactions:
        rxn.id = rxn.id+"2"
        rxn.name = rxn.name+" Night"

    #merge the day and night model
    cobra_model = cobra_model2.merge(cobra_model3)
    for met in cobra_model3.metabolites:
        if not cobra_model.metabolites.__contains__(met.id):
            cobra_model.add_metabolites(met.copy())
    for comp in cobra_model.compartments:
        cobra_model.compartments[comp] = tempCompDict[comp]

    met1 = Metabolite("X_Aracore_contribution_t1",name="biomass output during the day",compartment="b1")
    cobra_model.reactions.get_by_id("AraCore_Biomass_tx1").add_metabolites({met1:1})
    met2 = Metabolite("X_Aracore_contribution_t2",name="bimass output during at night",compartment="b1")
    cobra_model.reactions.get_by_id("AraCore_Biomass_tx2").add_metabolites({met2:1})

    rxn = Reaction("diel_biomass")
    rxn.add_metabolites({met1:-3,met2:-1})
    rxn.lower_bound = 0
    rxn.upper_bound = 1000
    cobra_model.add_reactions([rxn,])

    #Adding reactions to allow for day-night metabolite accumulations
    if transferMets!="":
        tmfile = open(transferMets,"r")
        tmset=set()
        for line in tmfile:
            tmset.add(line.replace("\n",""))
    else:
        tmset=set(["STARCH_p","SUCROSE_v","MAL_v","aMAL_v","NITRATE_v","CIT_v",
        "aCIT_v","GLN_v","ASN_v","SER_v","GLN_v","GLY_v","THR_v","L_ALPHA_ALANINE_v",
        "4_AMINO_BUTYRATE_v","VAL_v","ILE_v","PHE_v","LEU_v","LYS_v","ARG_v",
        "L_ASPARTATE_v","GLT_v","HIS_v","bHIS_v","MET_v","PRO_v","TRP_v","TYR_v",
        "CYS_v","FRUCTAN_v","AMMONIUM_v","PROTON_v"])

    for met in tmset:
        if met == "AMMONIUM_v" or met=="FRUCTAN_v":
            continue
        tempRxn = Reaction(met+"_dielTransfer")
        tempRxn.add_metabolites({cobra_model.metabolites.get_by_id(met+"1"):-1,cobra_model.metabolites.get_by_id(met+"2"):1})
        tempRxn.lower_bound=-1000
        if not ((met == "STARCH_p") or (met == "SUCROSE_v") or (met == "MAL_v") or (met == "aMAL_v") or (met == "NITRATE_v") or (met == "CIT_v") or (met == "aCIT_v") or (met == "PROTON_v")):
            tempRxn.lower_bound=0
        tempRxn.upper_bound=1000
        cobra_model.add_reactions([tempRxn,])

    fractionMets=dict()
    for rxn in cobra_model.reactions:
        for met in rxn.metabolites.keys():
            prefix=""
            a=re.search("^a{1,3}",met.id)
            anion=""
            if a:
                anion=a.group(0)
                prefix=anion
            b=re.search("^b{1,3}",met.id)
            basic=""
            if b:
                basic=b.group(0)
                prefix=basic
            if ((not prefix == "") and met.compartment == "v1"):
                fractionMets[met]=prefix

    temp=cobra_model.copy()
    for met in fractionMets.keys():
        for rxn in met.reactions:
            if rxn.id.__contains__("_dielTransfer"):
                continue
            else:
                mainMet = met.id[len(fractionMets[met]):]
                coeff1 = temp.reactions.get_by_id(rxn.id).metabolites.get(temp.metabolites.get_by_id(mainMet))
                coeff2 = temp.reactions.get_by_id(rxn.id).metabolites.get(temp.metabolites.get_by_id(met.id))
                if not coeff1:
                    coeff1=0
                if not coeff2:
                    coeff2=0
                total = coeff1 + coeff2
                coeff1 = float(coeff1)/total
                coeff2 = float(coeff2)/total
                if cobra_model.reactions.has_id(met.id[0:len(met.id)-1]+"_dielTransfer"):
                    ub = temp.reactions.get_by_id(met.id[0:len(met.id)-1]+"_dielTransfer").upper_bound
                    lb = temp.reactions.get_by_id(met.id[0:len(met.id)-1]+"_dielTransfer").lower_bound
                    temp.reactions.get_by_id(met.id[0:len(met.id)-1]+"_dielTransfer").remove_from_model()
                    temp.reactions.get_by_id(mainMet[0:len(mainMet)-1]+"_dielTransfer").remove_from_model()
                    Reac = Reaction(mainMet[0:len(mainMet)-1]+"_dielTransfer",name=mainMet+"_dielTransfer")
                    Reac.add_metabolites({temp.metabolites.get_by_id(met.id[0:len(met.id)-1]+"1"):-coeff2,temp.metabolites.get_by_id(met.id[0:len(met.id)-1]+"2"):coeff2,temp.metabolites.get_by_id(mainMet[0:len(mainMet)-1]+"1"):-coeff1,temp.metabolites.get_by_id(mainMet[0:len(mainMet)-1]+"2"):coeff1})
                    Reac.lower_bound=lb
                    Reac.upper_bound=ub
                    temp.add_reactions([Reac,])
                break
    ####ADD CONSTRAINTS TO MODEL####
    cobra_model = temp.copy()

    #objective function
    cobra_model.reactions.get_by_id("diel_biomass").objective_coefficient=1
    #Leaves - light
    cobra_model.reactions.get_by_id("Sucrose_tx1").lower_bound=0
    cobra_model.reactions.get_by_id("Sucrose_tx1").upper_bound=0
    cobra_model.reactions.get_by_id("GLC_tx1").lower_bound=0
    cobra_model.reactions.get_by_id("GLC_tx1").upper_bound=0
    cobra_model.reactions.get_by_id("NH4_tx1").lower_bound=0
    cobra_model.reactions.get_by_id("NH4_tx1").upper_bound=0
    #Leaves - dark
    cobra_model.reactions.get_by_id("Sucrose_tx2").lower_bound=0
    cobra_model.reactions.get_by_id("Sucrose_tx2").upper_bound=0
    cobra_model.reactions.get_by_id("GLC_tx2").lower_bound=0
    cobra_model.reactions.get_by_id("GLC_tx2").upper_bound=0
    cobra_model.reactions.get_by_id("Photon_tx2").lower_bound=0
    cobra_model.reactions.get_by_id("Photon_tx2").upper_bound=0
    cobra_model.reactions.get_by_id("NH4_tx2").lower_bound=0
    cobra_model.reactions.get_by_id("NH4_tx2").upper_bound=0

    #Turn off PTOX
    cobra_model.reactions.get_by_id("Plastoquinol_Oxidase_p1").lower_bound=0
    cobra_model.reactions.get_by_id("Plastoquinol_Oxidase_p1").upper_bound=0

    #nitrate uptake constrain
    Nitrate_balance = Metabolite("Nitrate_bal_c", name = "Weights to balance nitrate uptake", compartment = "c1")
    cobra_model.reactions.get_by_id("Nitrate_ec1").add_metabolites({Nitrate_balance:-2})
    cobra_model.reactions.get_by_id("Nitrate_ec2").add_metabolites({Nitrate_balance:3})

    #Rubisco balance
    Rubisco_balance = Metabolite("rubisco_bal_p1", name = "Weights to balance RuBP carboxygenase oxygenase balance", compartment = "p1")
    cobra_model.reactions.get_by_id("RXN_961_p1").add_metabolites({Rubisco_balance:3})
    cobra_model.reactions.get_by_id("RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p1").add_metabolites({Rubisco_balance:-1})

    #generic ATPase and NADPH oxidase
    Maintenance_constraint = Metabolite("ATPase_NADPHoxidase_constraint_c1",name =  "ATPase_NADPHoxidase_constraint_c1", compartment = "c1")
    Maintenance_constraint2 = Metabolite("ATPase_NADPHoxidase_constraint_c2",name =  "ATPase_NADPHoxidase_constraint_c2", compartment = "c2")
    Maintenance_constraint3 = Metabolite("Light_dark_maintainence_constraint",name =  "Light_dark_maintainence_constraint", compartment = "c1")
    cobra_model.reactions.get_by_id("ATPase_tx1").add_metabolites({Maintenance_constraint:1,Maintenance_constraint3:1})
    cobra_model.reactions.get_by_id("ATPase_tx2").add_metabolites({Maintenance_constraint2:1,Maintenance_constraint3:-1})
    cobra_model.reactions.get_by_id("NADPHoxc_tx1").add_metabolites({Maintenance_constraint:-3})
    cobra_model.reactions.get_by_id("NADPHoxc_tx2").add_metabolites({Maintenance_constraint2:-3})
    cobra_model.reactions.get_by_id("NADPHoxm_tx1").add_metabolites({Maintenance_constraint:-3})
    cobra_model.reactions.get_by_id("NADPHoxm_tx2").add_metabolites({Maintenance_constraint2:-3})
    cobra_model.reactions.get_by_id("NADPHoxp_tx1").add_metabolites({Maintenance_constraint:-3})
    cobra_model.reactions.get_by_id("NADPHoxp_tx2").add_metabolites({Maintenance_constraint2:-3})

    ##constrain sucrose and starch storage
    if starch_sucrose_ratio is not None:
        Sucorse_starch_balance = Metabolite("sucrose_starch_bal_c", name = "Weights to balance sucrose-starch uptake", compartment = "c1")
        cobra_model.reactions.get_by_id("SUCROSE_v_dielTransfer").add_metabolites({Sucorse_starch_balance:-1*starch_sucrose_ratio})
        cobra_model.reactions.get_by_id("STARCH_p_dielTransfer").add_metabolites({Sucorse_starch_balance:1})

    #Plastid enolase was not detected in Arabidopsis mesophyll tissue
    cobra_model.reactions.get_by_id("2PGADEHYDRAT_RXN_p1").lower_bound=0
    cobra_model.reactions.get_by_id("2PGADEHYDRAT_RXN_p1").upper_bound=0
    cobra_model.reactions.get_by_id("2PGADEHYDRAT_RXN_p2").lower_bound=0
    cobra_model.reactions.get_by_id("2PGADEHYDRAT_RXN_p2").upper_bound=0

    #Setting chloroplastic NADPH dehydrogenase to 0  ((Yamamoto et al., 2011)
    cobra_model.reactions.get_by_id("NADPH_Dehydrogenase_p1").lower_bound=0
    cobra_model.reactions.get_by_id("NADPH_Dehydrogenase_p1").upper_bound=0
    cobra_model.reactions.get_by_id("NADPH_Dehydrogenase_p2").lower_bound=0
    cobra_model.reactions.get_by_id("NADPH_Dehydrogenase_p2").upper_bound=0

    #ATP_ADP_Pi constrained to 0 because while there is evidence for its existance, it does not carry high flux
    cobra_model.reactions.get_by_id("ATP_ADP_Pi_pc1").lower_bound = 0
    cobra_model.reactions.get_by_id("ATP_ADP_Pi_pc1").upper_bound = 0
    cobra_model.reactions.get_by_id("ATP_ADP_Pi_pc2").lower_bound = 0
    cobra_model.reactions.get_by_id("ATP_ADP_Pi_pc2").upper_bound = 0

    print(cobra_model.reactions.get_by_id("AraCore_Biomass_tx1").bounds)
    print(cobra_model.reactions.get_by_id("AraCore_Biomass_tx2").bounds)

    return cobra_model


# Core Model
## loading the PlantCoreMetabolism model

In [ ]:
core_model = read_sbml_model("/content/drive/My Drive/protocolFBA/models/PlantCoreMetabolism_v2_0_0.xml")

##making diel model
### List of models

*   diel_model0 = diel leaf model of plantCoreMetabolism model
*   diel_model1 = diel leaf model of plantCoreMetabolism model with biomass composition of AraCore
*   diel_model2 = diel leaf model of plantCoreMetabolism model with biomass composition of AraGEM
*   diel_model3 = diel leaf model of plantCoreMetabolism model with biomass composition of AraMeta





In [ ]:
diel_model0 = setupC3DielModel(core_model,transferMets= "", starch_sucrose_ratio =None)

(0.0, 1000.0)
(0.0, 1000.0)


In [ ]:
diel_model0.objective = diel_model0.reactions.get_by_id("diel_biomass")
solution0 = pfba(diel_model0)
diel_model0

Name,PlantCoreMetabolism_v2_0_0
Memory address,7da0eccf1910
Number of metabolites,1769
Number of reactions,1874
Number of genes,0
Number of groups,208
Objective expression,1.0*diel_biomass - 1.0*diel_biomass_reverse_79408
Compartments,"m1, c1, b1, p1, v1, x1, r1, mi1, mc1, e1, l1, i1, c2, v2, e2, x2, b2, p2, m2, mi2, i2, mc2, r2, l2"


In [ ]:
# constraining GPT transport reaction
diel_model0.reactions.get_by_id("G6P_Pi_pc1").lower_bound= 0
diel_model0.reactions.get_by_id("G6P_Pi_pc1").upper_bound= 0
diel_model0.reactions.get_by_id("G6P_Pi_pc2").lower_bound= 0
diel_model0.reactions.get_by_id("G6P_Pi_pc2").upper_bound= 0
# constraining starch phosphorylation
diel_model0.reactions.get_by_id("RXN_1826_p1").lower_bound= 0
diel_model0.reactions.get_by_id("RXN_1826_p1").upper_bound= 0
diel_model0.reactions.get_by_id("RXN_1826_p2").lower_bound= 0
diel_model0.reactions.get_by_id("RXN_1826_p2").upper_bound= 0
# Setting photon uptake
PPFD = 500
diel_model0.reactions.Photon_tx1.upper_bound = PPFD
diel_model0.reactions.Photon_tx1.lower_bound = 0
# Setting maintenance cost
VATPase = 0.0049*PPFD+2.7851
diel_model0.reactions.get_by_id("ATPase_tx1").bounds = (VATPase,VATPase)
# constraining sucrose accumulation to model a starch storing leaf
diel_model0.reactions.get_by_id("SUCROSE_v_dielTransfer").bounds = (0,0)
solution0 = pfba(diel_model0)

In [ ]:
diel_model0.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
CARBON_DIOXIDE_e1,CO2_tx1,31.98,1,100.00%
CAII_e1,Ca_tx1,0.2242,0,0.00%
CAII_e2,Ca_tx2,0.07475,0,0.00%
WATER_e1,H2O_tx1,27.7,0,0.00%
KI_e1,K_tx1,0.4587,0,0.00%
KI_e2,K_tx2,0.1529,0,0.00%
MGII_e1,Mg_tx1,0.1478,0,0.00%
MGII_e2,Mg_tx2,0.04926,0,0.00%
NITRATE_e1,Nitrate_tx1,1.55,0,0.00%
NITRATE_e2,Nitrate_tx2,1.033,0,0.00%


#AraCoremodel
## applying biomass composition from AcarCore model by Arnold & Nikoloski,2014
###(the sbml file is updated by adding the respective biomass composition)


#Loading the model from github

In [ ]:
# Core_AraCoremodel = read_sbml_model("Core_AraCore.xml")       # Sanus version
Core_AraCoremodel = read_sbml_model("/content/drive/My Drive/protocolFBA/models/Core_AraCore.xml")     # Vipinas version

In [ ]:
diel_model1 = setupC3DielModel(Core_AraCoremodel,transferMets= "", starch_sucrose_ratio =None)

(0.0, 1000.0)
(0.0, 1000.0)


##resetting objective and optimization

In [ ]:
diel_model1.objective = diel_model1.reactions.get_by_id("diel_biomass")
solution1 = pfba(diel_model1)
diel_model1

Name,PlantCoreMetabolism_v2_0_0
Memory address,7da0ea829590
Number of metabolites,1769
Number of reactions,1874
Number of genes,0
Number of groups,208
Objective expression,1.0*diel_biomass - 1.0*diel_biomass_reverse_79408
Compartments,"m1, c1, b1, p1, v1, x1, r1, mi1, mc1, e1, l1, i1, c2, v2, e2, x2, b2, p2, m2, mi2, i2, mc2, r2, l2"


#constraining model into C3 with new biomass composition

In [ ]:
# constraining GPT transport reaction
diel_model1.reactions.get_by_id("G6P_Pi_pc1").lower_bound= 0
diel_model1.reactions.get_by_id("G6P_Pi_pc1").upper_bound= 0
diel_model1.reactions.get_by_id("G6P_Pi_pc2").lower_bound= 0
diel_model1.reactions.get_by_id("G6P_Pi_pc2").upper_bound= 0
# constraining starch phosphorylation
diel_model1.reactions.get_by_id("RXN_1826_p1").lower_bound= 0
diel_model1.reactions.get_by_id("RXN_1826_p1").upper_bound= 0
diel_model1.reactions.get_by_id("RXN_1826_p2").lower_bound= 0
diel_model1.reactions.get_by_id("RXN_1826_p2").upper_bound= 0
# Setting photon uptake
PPFD = 500
diel_model1.reactions.Photon_tx1.upper_bound = PPFD
diel_model1.reactions.Photon_tx1.lower_bound = 0
# Setting maintenance cost
VATPase = 0.0049*PPFD+2.7851
diel_model1.reactions.get_by_id("ATPase_tx1").bounds = (VATPase,VATPase)
# constraining sucrose accumulation to model a starch storing leaf
diel_model1.reactions.get_by_id("SUCROSE_v_dielTransfer").bounds = (0,0)
solution1 = pfba(diel_model1)

In [ ]:
diel_model1.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
CARBON_DIOXIDE_e1,CO2_tx1,32.29,1,100.00%
WATER_e1,H2O_tx1,26.29,0,0.00%
NITRATE_e1,Nitrate_tx1,2.777,0,0.00%
NITRATE_e2,Nitrate_tx2,1.851,0,0.00%
OXYGEN_MOLECULE_e2,O2_tx2,2.717,0,0.00%
Photon_e1,Photon_tx1,500,0,0.00%
Pi_e1,Pi_tx1,0.01284,0,0.00%
Pi_e2,Pi_tx2,0.004279,0,0.00%
SULFATE_e1,SO4_tx1,0.1141,0,0.00%
PROTON_c1,unlProtHYPO_c1,2.826,0,0.00%


#AraGEMmodel
## applying biomass composition from AcarCore model by de Oliveira Dal'Molin et al.,2010
###(the sbml file is updated by adding the respective biomass composition)
Loading the model

In [ ]:
Core_AraGEMmodel = read_sbml_model("/content/drive/My Drive/protocolFBA/models/Core_AraGEM.xml")

In [ ]:
diel_model2 = setupC3DielModel(Core_AraGEMmodel,transferMets= "", starch_sucrose_ratio =None)

(0.0, 1000.0)
(0.0, 1000.0)


In [ ]:
diel_model2.objective = diel_model2.reactions.get_by_id("diel_biomass")
solution2 = pfba(diel_model2)
diel_model2

Name,PlantCoreMetabolism_v2_0_0
Memory address,7da0e9fa7890
Number of metabolites,1777
Number of reactions,1874
Number of genes,0
Number of groups,208
Objective expression,1.0*diel_biomass - 1.0*diel_biomass_reverse_79408
Compartments,"m1, c1, b1, p1, v1, x1, r1, mi1, mc1, e1, l1, i1, c2, v2, e2, x2, b2, p2, m2, mi2, i2, mc2, r2, l2"


In [ ]:
# constraining GPT transport reaction
diel_model2.reactions.get_by_id("G6P_Pi_pc1").lower_bound= 0
diel_model2.reactions.get_by_id("G6P_Pi_pc1").upper_bound= 0
diel_model2.reactions.get_by_id("G6P_Pi_pc2").lower_bound= 0
diel_model2.reactions.get_by_id("G6P_Pi_pc2").upper_bound= 0
# constraining starch phosphorylation
diel_model2.reactions.get_by_id("RXN_1826_p1").lower_bound= 0
diel_model2.reactions.get_by_id("RXN_1826_p1").upper_bound= 0
diel_model2.reactions.get_by_id("RXN_1826_p2").lower_bound= 0
diel_model2.reactions.get_by_id("RXN_1826_p2").upper_bound= 0
# Setting photon uptake
PPFD = 500
diel_model2.reactions.Photon_tx1.upper_bound = PPFD
diel_model2.reactions.Photon_tx1.lower_bound = 0
# Setting maintenance cost
VATPase = 0.0049*PPFD+2.7851
diel_model2.reactions.get_by_id("ATPase_tx1").bounds = (VATPase,VATPase)
# constraining sucrose accumulation to model a starch storing leaf
diel_model2.reactions.get_by_id("SUCROSE_v_dielTransfer").bounds = (0,0)
solution2 = pfba(diel_model2)

In [ ]:
diel_model2.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
CARBON_DIOXIDE_e1,CO2_tx1,29.11,1,100.00%
WATER_e1,H2O_tx1,25.05,0,0.00%
NITRATE_e1,Nitrate_tx1,2.746,0,0.00%
NITRATE_e2,Nitrate_tx2,1.831,0,0.00%
OXYGEN_MOLECULE_e2,O2_tx2,4.383,0,0.00%
Photon_e1,Photon_tx1,500,0,0.00%
Pi_e1,Pi_tx1,0.03829,0,0.00%
Pi_e2,Pi_tx2,0.01276,0,0.00%
SULFATE_e1,SO4_tx1,0.03735,0,0.00%
PROTON_c1,unlProtHYPO_c1,3.842,0,0.00%


#AraMetamodel
## applying biomass composition from AcarCore model by Poolman et al.,2009
###(the sbml file is updated by adding the respective biomass composition)
Loading the model

In [ ]:
Core_AraMetamodel = read_sbml_model("/content/drive/My Drive/protocolFBA/models/Core_AraMeta.xml")

In [ ]:
diel_model3= setupC3DielModel(Core_AraMetamodel,transferMets= "", starch_sucrose_ratio =None)

(0.0, 1000.0)
(0.0, 1000.0)


In [ ]:
diel_model3.objective = diel_model3.reactions.get_by_id("diel_biomass")
solution3 = pfba(diel_model3)
diel_model3

Name,PlantCoreMetabolism_v2_0_0
Memory address,7da0ea758e90
Number of metabolites,1773
Number of reactions,1874
Number of genes,0
Number of groups,208
Objective expression,1.0*diel_biomass - 1.0*diel_biomass_reverse_79408
Compartments,"m1, c1, b1, p1, v1, x1, r1, mi1, mc1, e1, l1, i1, c2, v2, e2, x2, b2, p2, m2, mi2, i2, mc2, r2, l2"


In [ ]:
# constraining GPT transport reaction
diel_model3.reactions.get_by_id("G6P_Pi_pc1").lower_bound= 0
diel_model3.reactions.get_by_id("G6P_Pi_pc1").upper_bound= 0
diel_model3.reactions.get_by_id("G6P_Pi_pc2").lower_bound= 0
diel_model3.reactions.get_by_id("G6P_Pi_pc2").upper_bound= 0
# constraining starch phosphorylation
diel_model3.reactions.get_by_id("RXN_1826_p1").lower_bound= 0
diel_model3.reactions.get_by_id("RXN_1826_p1").upper_bound= 0
diel_model3.reactions.get_by_id("RXN_1826_p2").lower_bound= 0
diel_model3.reactions.get_by_id("RXN_1826_p2").upper_bound= 0
# Setting photon uptake
PPFD = 500
diel_model3.reactions.Photon_tx1.upper_bound = PPFD
diel_model3.reactions.Photon_tx1.lower_bound = 0
# Setting maintenance cost
VATPase = 0.0049*PPFD+2.7851
diel_model3.reactions.get_by_id("ATPase_tx1").bounds = (VATPase,VATPase)
# constraining sucrose accumulation to model a starch storing leaf
diel_model3.reactions.get_by_id("SUCROSE_v_dielTransfer").bounds = (0,0)
solution3 = pfba(diel_model3)

In [ ]:
diel_model3.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
CARBON_DIOXIDE_e1,CO2_tx1,29.51,1,100.00%
WATER_e1,H2O_tx1,26.1,0,0.00%
NITRATE_e1,Nitrate_tx1,3.086,0,0.00%
NITRATE_e2,Nitrate_tx2,2.057,0,0.00%
OXYGEN_MOLECULE_e2,O2_tx2,3.178,0,0.00%
Photon_e1,Photon_tx1,500,0,0.00%
Pi_e1,Pi_tx1,0.1977,0,0.00%
Pi_e2,Pi_tx2,0.06589,0,0.00%
SULFATE_e1,SO4_tx1,0.07924,0,0.00%
PROTON_c1,unlProtHYPO_c1,4.342,0,0.00%


#Comparing the results by percentage

In [ ]:
import pandas as pd
reaction_ids = ["CO2_tx1", "CO2_tx2","CO2_pc1","RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p1","PHOSPHORIBULOKINASE_RXN_p1","RXN_961_p1","F16ALDOLASE_RXN_p1","GLYCOGENSYN_RXN_p1","FERREDOXIN_NITRITE_REDUCTASE_RXN_p1",
                "ASPAMINOTRANS_RXN_c1","L_ASPARTATE_pc1","GLY3KIN_RXN_p1","GLYCERATE_GLYCOLLATE_pc1","GPH_RXN_p1","GLYCOLLATE_pc1","2KG_MAL_pc1","GAP_Pi_pc1","PEPCARBOX_RXN_c1","Nitrate_ec1","NITRATE_vc1",
                "FERREDOXIN_NITRITE_REDUCTASE_RXN_p1","Glycolate_xc1","RXN_969_x1","GLYCINE_AMINOTRANSFERASE_RXN_x1","SERINE_GLYOXYLATE_AMINOTRANSFERASE_RXN_x1","GCVMULTI_RXN_m1","GLYOHMETRANS_RXN_m1",
                "HYDROXYPYRUVATE_REDUCTASE_RXN_NAD_x1","AraCore_Biomass_tx1","F16BDEPHOS_RXN_c1","Starch_biomass1","RXN_1827_p2","MALTODEG_RXN_c2","GLU6PDEHYDROG_RXN_c2","GAPOXNPHOSPHN_RXN_c2",
                "AraCore_Biomass_tx2","2PGADEHYDRAT_RXN_c2","PEPDEPHOS_RXN_c2","PYRUVDEH_RXN_m2","MALATE_DEH_RXN_m2","CITSYN_RXN_m2","STARCH_p_dielTransfer", "MAL_v_dielTransfer","CIT_v_dielTransfer","NITRATE_v_dielTransfer"]

# to take all common reactions from solution4 (reference)
# reaction_ids = solution4.fluxes.index.tolist()

flux_comparison = pd.DataFrame({
    "Reaction": reaction_ids,
    "Core_AraCore": [solution1.fluxes.get(rxn, 0.0) for rxn in reaction_ids],
    "Core_AraGEM": [solution2.fluxes.get(rxn, 0.0) for rxn in reaction_ids],
    "Core_AraMeta": [solution3.fluxes.get(rxn, 0.0) for rxn in reaction_ids],
    "Core_model": [solution4.fluxes.get(rxn, 0.0) for rxn in reaction_ids]
})
flux_comparison["% Diff (Core_AraCore)"] = 100 * (flux_comparison["Core_AraCore"] - flux_comparison["Core_model"]) / flux_comparison["Core_model"]
flux_comparison["% Diff (Core_AraGEM)"]   = 100 * (flux_comparison["Core_AraGEM"]   - flux_comparison["Core_model"]) / flux_comparison["Core_model"]
flux_comparison["% Diff (Core_AraMeta)"]  = 100 * (flux_comparison["Core_AraMeta"]  - flux_comparison["Core_model"]) / flux_comparison["Core_model"]

#comparing results with Log2Fchange

In [ ]:
import pandas as pd
import numpy as np


for model in ["Core_AraCore", "Core_AraGEM", "Core_AraMeta"]:
    num = flux_comparison[model]
    den = flux_comparison["Core_model"]

    log2fc = []

    for n, d in zip(num, den):
        if n == 0 or d == 0:
            log2fc.append(0)  # If either flux is zero → set log2FC to 0
        elif (n / d) < 0:
            log2fc.append(np.nan)  # If ratio is negative → invalid → set to NaN
        else:
            log2fc.append(np.log2(n / d))  # Valid ratio → compute log2FC

    flux_comparison[f"Log2FC_{model}"] = log2fc

#     # Save to Excel
# output_file = "flux_comparison_log2FC_1.xlsx"
# flux_comparison.to_excel(output_file, index=False)
